## Customer Churn Prediction - Logistic Regression

In [1]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sns

### 3.1 Data Preparation
- Download the data and read with pandas
- Look at the data
- Unified the format of columns name and values for categorical columns
- Investigate missing values
- Check if the churn variable needs any cleaning

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
# Convert columns name to lower case and replace space with underscore
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Identify categorical columns in the dataframe
categorical_col = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_col:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [5]:
# View the records of long table in column format
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   seniorcitizen     7043 non-null   int64  
 3   partner           7043 non-null   object 
 4   dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   phoneservice      7043 non-null   object 
 7   multiplelines     7043 non-null   object 
 8   internetservice   7043 non-null   object 
 9   onlinesecurity    7043 non-null   object 
 10  onlinebackup      7043 non-null   object 
 11  deviceprotection  7043 non-null   object 
 12  techsupport       7043 non-null   object 
 13  streamingtv       7043 non-null   object 
 14  streamingmovies   7043 non-null   object 
 15  contract          7043 non-null   object 
 16  paperlessbilling  7043 non-null   object 


totalcharges column is still read as a text columns. Need to clean and convert to numerical format. The reason is that there are values in this column which has been ommited and replaced with '_'. Pandas cannot convert this to numeric value. Will need to replace them with 0.

In [7]:
df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
df['totalcharges'] = df['totalcharges'].fillna(0)

In [8]:
df['totalcharges'].isnull().sum()

np.int64(0)

In [9]:
# Inspect the 'churn' variable
df['churn'].value_counts()

churn
no     5174
yes    1869
Name: count, dtype: int64

In [10]:
# Convert the churn value from yes/no to numeric 1/0
df['churn'] = (df['churn'] == 'yes').astype(int)

In [11]:
df['churn'].value_counts()

churn
0    5174
1    1869
Name: count, dtype: int64

#### 3.2 Setting up validation framework
Using sci-kit learn package to split train/validation/test data

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

# further split the train dataset into train and validation
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [14]:
len(df_train), len(df_val), len(df_test)

(4225, 1409, 1409)

In [15]:
y_train = df_train['churn'].values
y_val = df_val['churn'].values
y_test = df_test['churn'].values

In [16]:
del df_train['churn']
del df_val['churn']
del df_test['churn']

#### 3.3 EDA
- Check for missing value
- Look at the target variable
- Look at numerical and categorical variables

In [17]:
df_full_train = df_full_train.reset_index(drop=True)

Check the train data set for any missing values. There are no columns with missing values so we dont have to do any cleaning step here.

In [18]:
df_full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

Next, we investigate the target variable 'churn'. The split is roughly 70:30. The churn rate is about 27%.

In [19]:
df_full_train['churn'].value_counts(normalize=True)

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

Next, we split the numerical columns from categorical columns.

In [20]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']
categorical = [col for col in df_full_train.columns if col not in numerical]
categorical.remove('churn')
categorical.remove('customerid')

In [21]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

#### 3.4 Feature Importance: Churn Rate and Risk Ratio

1. *Churn rate*: Difference between the global mean of the target variable (global churn rate) and mean of the target variable for categories of a feature. If this difference is greater than 0, it means that this group is more likely to churn and vice versa. The larger the difference, the more important/useful the categorical variable can be used in predicting the churn rate.

2. *Risk ratio*: Ratio between the mean of the target variable for caegories of a feature and global mean of target variable. If this ratio is greater than 1, the category is more likely to churn, and vice versa. It is another way of expressing the feature importance / usefulness in predicting churn rate.

In [22]:
global_churn = df_full_train['churn'].mean()
global_churn

np.float64(0.26996805111821087)

In [23]:
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean()
churn_no_partner

np.float64(0.3298090040927694)

In [24]:
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_partner

np.float64(0.20503330866025166)

In [25]:
global_churn - churn_partner

np.float64(0.06493474245795922)

In [26]:
churn_partner/global_churn

np.float64(0.7594724924338315)

In [27]:
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c)['churn'].agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)
    print()
    print()

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980




seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208




partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730




paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560




paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


#### 3.5 Feature importance: Mutual Information
Mutual information is a concept from information theory. It tells us how much we can learn about one variable if we know the value of other variable. We can use sklearn package to calculate the mutual information score bewteen the target variable and other categorical variables to find out which feature is more important in predicting the target.

In [29]:
from sklearn.metrics import mutual_info_score

In [31]:
# Calculate the mi score between churn and gender
# Similar to the churn rate difference based on gender, the mi score between gender and churn is very low, showing that gender will not be useful in predicting churn rate
mutual_info_score(df_full_train.gender, df_full_train.churn)

0.0001174846211139946

In [32]:
# The mi score between churn and contractype is much more significant
mutual_info_score(df_full_train.contract, df_full_train.churn)

0.0983203874041556

In [37]:
# Interestingly, the mi score for churn itself is not 1 but only 0.58. 
# However, this is significantly high by 1 order of magnitude comapred to contracttype
mutual_info_score(df_full_train.churn, df_full_train.churn)

0.5832270604600812

In [38]:
def mi_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [39]:
mi = df_full_train[categorical].apply(mi_churn_score)
mi.sort_values(ascending=False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

In the order of improtance, contract, online security, techsupport for examples are more important and provide more information in predicting churn rate. In the opposite, gender and phoneservice offer very little information for churn prediction.

#### 3.6 Feature Importance: Correlation
Correlation coefficient is another way to show how variables move relatively to each other. The coefficient is between -1 <= r <= 1. A positive r means two variables moves in the same direction. When r = 0, it means that two variables do no move together and not correlated (independent).

In [41]:
df_full_train[numerical].corrwith(df_full_train.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

Tenure and churn has correlation coefficient of -0.35. 